Using a dataset of your choice, select an outcome variable and then pick four or five other variables (one to two categorical, three to four continuous) to act as the basis for features. Explore the variables using the univariate and bivariate methods you've learned so far. 

Next, based on what you learned via your data exploration, create ten new features. Explain the reasoning behind each one.

Finally, use filtering methods to select the five best features and justify your choices.

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
%matplotlib inline

from GDELT_utils import GDELT_columns, usecols, dtype_dict, \
                        cameo_dict, map_cameo_to_text, \
                        state_dict, mem_usage, state_heat_map
        
path = "D:/GDELT-files-trim/"
nov_2016 = [path + "20161101.export.CSV.pkl", path + "20161111.export.CSV.pkl", path + "20161121.export.CSV.pkl",
            path + "20161102.export.CSV.pkl", path + "20161112.export.CSV.pkl", path + "20161122.export.CSV.pkl",
            path + "20161103.export.CSV.pkl", path + "20161113.export.CSV.pkl", path + "20161123.export.CSV.pkl",
            path + "20161104.export.CSV.pkl", path + "20161114.export.CSV.pkl", path + "20161124.export.CSV.pkl",
            path + "20161105.export.CSV.pkl", path + "20161115.export.CSV.pkl", path + "20161125.export.CSV.pkl",
            path + "20161106.export.CSV.pkl", path + "20161116.export.CSV.pkl", path + "20161126.export.CSV.pkl",
            path + "20161107.export.CSV.pkl", path + "20161117.export.CSV.pkl", path + "20161127.export.CSV.pkl",
            path + "20161108.export.CSV.pkl", path + "20161118.export.CSV.pkl", path + "20161128.export.CSV.pkl",
            path + "20161109.export.CSV.pkl", path + "20161119.export.CSV.pkl", path + "20161129.export.CSV.pkl",
            path + "20161110.export.CSV.pkl", path + "20161120.export.CSV.pkl", path + "20161130.export.CSV.pkl"]

path = "D:/GDELT-files/events/"
nov_2016 = [path + "20161101.export.CSV.zip", path + "20161111.export.CSV.zip", path + "20161121.export.CSV.zip",
            path + "20161102.export.CSV.zip", path + "20161112.export.CSV.zip", path + "20161122.export.CSV.zip",
            path + "20161103.export.CSV.zip", path + "20161113.export.CSV.zip", path + "20161123.export.CSV.zip",
            path + "20161104.export.CSV.zip", path + "20161114.export.CSV.zip", path + "20161124.export.CSV.zip",
            path + "20161105.export.CSV.zip", path + "20161115.export.CSV.zip", path + "20161125.export.CSV.zip",
            path + "20161106.export.CSV.zip", path + "20161116.export.CSV.zip", path + "20161126.export.CSV.zip",
            path + "20161107.export.CSV.zip", path + "20161117.export.CSV.zip", path + "20161127.export.CSV.zip",
            path + "20161108.export.CSV.zip", path + "20161118.export.CSV.zip", path + "20161128.export.CSV.zip",
            path + "20161109.export.CSV.zip", path + "20161119.export.CSV.zip", path + "20161129.export.CSV.zip",
            path + "20161110.export.CSV.zip", path + "20161120.export.CSV.zip", path + "20161130.export.CSV.zip"]

In [38]:
interest = ["GLOBALEVENTID", "SQLDATE",
            "AvgTone", "QuadClass", "GoldsteinScale",
            "NumMentions", "IsRootEvent", "EventRootCode",
            #"EventBaseCode", "EventCode",
            #"Actor1Code", "Actor2Code",
            "Actor1CountryCode", "Actor2CountryCode",
            "Actor1KnownGroupCode", "Actor2KnownGroupCode",
            #"Actor1EthnicCode", "Actor2EthnicCode",
            "Actor1Religion1Code", "Actor2Religion1Code",
            #"Actor1Religion2Code", "Actor2Religion2Code",
            "Actor1Type1Code", "Actor2Type1Code",
            #"Actor1Type2Code", "Actor2Type2Code",
            #"Actor1Type3Code", "Actor2Type3Code",
            "Actor1Geo_CountryCode", "Actor2Geo_CountryCode", "ActionGeo_CountryCode",
            #"Actor1Geo_ADM1Code", "Actor2Geo_ADM1Code", "ActionGeo_ADM1Code"
           ]

In [39]:
categories = [ "EventRootCode",
            "Actor1CountryCode", "Actor2CountryCode",
            "Actor1KnownGroupCode", "Actor2KnownGroupCode",
            "Actor1Religion1Code", "Actor2Religion1Code",
            "Actor1Type1Code", "Actor2Type1Code",
            "Actor1Geo_CountryCode", "Actor2Geo_CountryCode", "ActionGeo_CountryCode",
           ]

In [40]:
dfs = []
for file in nov_2016:
    df = pd.read_csv(file, sep='\t', names=GDELT_columns, 
                        usecols=interest, dtype=dtype_dict, 
                        parse_dates=['SQLDATE'])
    dfs.append(df)

In [41]:
dfs[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226375 entries, 0 to 226374
Data columns (total 19 columns):
GLOBALEVENTID            226375 non-null uint32
SQLDATE                  226375 non-null datetime64[ns]
Actor1CountryCode        125537 non-null category
Actor1KnownGroupCode     2838 non-null category
Actor1Religion1Code      3680 non-null category
Actor1Type1Code          100103 non-null category
Actor2CountryCode        99005 non-null category
Actor2KnownGroupCode     2185 non-null category
Actor2Religion1Code      3671 non-null category
Actor2Type1Code          81567 non-null category
IsRootEvent              226375 non-null bool
EventRootCode            226375 non-null category
QuadClass                226375 non-null uint8
GoldsteinScale           226370 non-null float32
NumMentions              226375 non-null uint16
AvgTone                  226375 non-null float32
Actor1Geo_CountryCode    200003 non-null category
Actor2Geo_CountryCode    160515 non-null category
Action

In [42]:
nov_2016 = pd.concat(dfs)

In [45]:
nov_2016.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6032301 entries, 0 to 234621
Data columns (total 19 columns):
GLOBALEVENTID            uint32
SQLDATE                  datetime64[ns]
Actor1CountryCode        object
Actor1KnownGroupCode     object
Actor1Religion1Code      object
Actor1Type1Code          object
Actor2CountryCode        object
Actor2KnownGroupCode     object
Actor2Religion1Code      object
Actor2Type1Code          object
IsRootEvent              bool
EventRootCode            object
QuadClass                uint8
GoldsteinScale           float32
NumMentions              uint16
AvgTone                  float32
Actor1Geo_CountryCode    object
Actor2Geo_CountryCode    object
ActionGeo_CountryCode    object
dtypes: bool(1), datetime64[ns](1), float32(2), object(12), uint16(1), uint32(1), uint8(1)
memory usage: 3.2 GB


In [46]:
for category_col in categories:
    nov_2016[category_col] = nov_2016[category_col].astype('category')

In [47]:
nov_2016.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6032301 entries, 0 to 234621
Data columns (total 19 columns):
GLOBALEVENTID            uint32
SQLDATE                  datetime64[ns]
Actor1CountryCode        category
Actor1KnownGroupCode     category
Actor1Religion1Code      category
Actor1Type1Code          category
Actor2CountryCode        category
Actor2KnownGroupCode     category
Actor2Religion1Code      category
Actor2Type1Code          category
IsRootEvent              bool
EventRootCode            category
QuadClass                uint8
GoldsteinScale           float32
NumMentions              uint16
AvgTone                  float32
Actor1Geo_CountryCode    category
Actor2Geo_CountryCode    category
ActionGeo_CountryCode    category
dtypes: bool(1), category(12), datetime64[ns](1), float32(2), uint16(1), uint32(1), uint8(1)
memory usage: 282.0 MB


In [48]:
nov_2016.shape

(6032301, 19)

In [49]:
nov_2016.corr()

,GLOBALEVENTID,IsRootEvent,QuadClass,GoldsteinScale,NumMentions,AvgTone
GLOBALEVENTID,1.000000,-0.003546,-0.002889,0.005619,-0.003755,-0.007180
IsRootEvent,-0.003546,1.000000,0.008951,-0.011832,0.040962,0.049632
QuadClass,-0.002889,0.008951,1.000000,-0.774402,0.008736,-0.358945
GoldsteinScale,0.005619,-0.011832,-0.774402,1.000000,-0.013464,0.347634
NumMentions,-0.003755,0.040962,0.008736,-0.013464,1.000000,-0.030566
AvgTone,-0.007180,0.049632,-0.358945,0.347634,-0.030566,1.000000


In [50]:
nov_2016.groupby('QuadClass').mean()

,GLOBALEVENTID,IsRootEvent,GoldsteinScale,NumMentions,AvgTone
QuadClass,,,,,
1,5.992662e+08,0.641602,2.445738,14.243583,-0.894462
2,5.992519e+08,0.635813,5.223845,14.244955,-1.607522
3,5.992091e+08,0.613754,-3.498103,14.489701,-2.973222
4,5.992672e+08,0.671940,-7.971389,16.707975,-4.698138


In [51]:
nov_2016.groupby('QuadClass').corr()

AvgTone  GLOBALEVENTID  GoldsteinScale  \
QuadClass                                                           
1         AvgTone         1.000000      -0.011323        0.152310   
          GLOBALEVENTID  -0.011323       1.000000        0.009457   
          GoldsteinScale  0.152310       0.009457        1.000000   
          IsRootEvent     0.109211      -0.003680        0.004957   
          NumMentions    -0.026461      -0.003557       -0.010022   
2         AvgTone         1.000000       0.003155        0.169980   
          GLOBALEVENTID   0.003155       1.000000        0.021554   
          GoldsteinScale  0.169980       0.021554        1.000000   
          IsRootEvent     0.065522      -0.002520       -0.040317   
          NumMentions    -0.031981      -0.000478       -0.016452   
3         AvgTone         1.000000      -0.004713       -0.001731   
          GLOBALEVENTID  -0.004713       1.000000       -0.004287   
          GoldsteinScale -0.001731      -0.004287        1.000000   
          IsRootEvent    -0.024759      -0.001176        0.000863   
          NumMentions    -0.029209      -0.005883       -0.007326   
4         AvgTone         1.000000      -0.009727        0.006737   
          GLOBALEVENTID  -0.009727       1.000000       -0.002073   
          GoldsteinScale  0.006737      -0.002073        1.000000   
          IsRootEvent    -0.084060      -0.007268        0.015516   
          NumMentions    -0.036946      -0.005315       -0.015070   

                          IsRootEvent  NumMentions  
QuadClass                                           
1         AvgTone            0.109211    -0.026461  
          GLOBALEVENTID     -0.003680    -0.003557  
          GoldsteinScale     0.004957    -0.010022  
          IsRootEvent        1.000000     0.043104  
          NumMentions        0.043104     1.000000  
2         AvgTone            0.065522    -0.031981  
          GLOBALEVENTID     -0.002520    -0.000478  
          GoldsteinScale    -0.040317    -0.016452  
          IsRootEvent        1.000000     0.027455  
          NumMentions        0.027455     1.000000  
3         AvgTone           -0.024759    -0.029209  
          GLOBALEVENTID     -0.001176    -0.005883  
          GoldsteinScale     0.000863    -0.007326  
          IsRootEvent        1.000000     0.040817  
          NumMentions        0.040817     1.000000  
4         AvgTone           -0.084060    -0.036946  
          GLOBALEVENTID     -0.007268    -0.005315  
          GoldsteinScale     0.015516    -0.015070  
          IsRootEvent        1.000000     0.041158  
          NumMentions        0.041158     1.000000

In [ ]:
groupby Day EventRootCode ActionGeo_CountryCode

In [52]:
nov_2016['Conflict'] = nov_2016['QuadClass'] > 2

In [53]:
nov_2016['Conflict'].value_counts()

False    4419418
True     1612883
Name: Conflict, dtype: int64

In [54]:
nov_2016['Violence'] = nov_2016['QuadClass'] == 4

In [55]:
nov_2016['Violence'].value_counts()

False    5196452
True      835849
Name: Violence, dtype: int64

In [56]:
nov_2016['GoldsteinTone'] = nov_2016['GoldsteinScale'] + nov_2016['AvgTone']

In [57]:
nov_2016.groupby('QuadClass').mean()

,GLOBALEVENTID,IsRootEvent,GoldsteinScale,NumMentions,AvgTone,Conflict,Violence,GoldsteinTone
QuadClass,,,,,,,,
1,5.992662e+08,0.641602,2.445738,14.243583,-0.894462,0.0,0.0,1.551282
2,5.992519e+08,0.635813,5.223845,14.244955,-1.607522,0.0,0.0,3.616324
3,5.992091e+08,0.613754,-3.498103,14.489701,-2.973222,1.0,0.0,-6.471289
4,5.992672e+08,0.671940,-7.971389,16.707975,-4.698138,1.0,1.0,-12.669526


In [29]:
from scipy.stats import f_oneway

In [ ]:

f_oneway()